# 完整Gradio界面 - Industrial Digital Twin by Transformer

## 📖 使用说明

本Notebook提供完整的Gradio网页界面，用于训练和推理V1和V4 Transformer模型。

### 🎯 两种使用方式

#### 方式A: 使用已提供的函数（简化版）
- 运行下面所有cells
- 启动Gradio界面
- 适合快速开始

#### 方式B: 集成您的完整Cell 3代码（完整功能）
- 运行Cell 1和Cell 2的导入
- 在标记的位置粘贴您原始的Cell 3代码
- 获得100%原始功能

### 📌 Cell 3集成位置

如果您有原始的 `说明.txt` Cell 3代码：
1. 运行Cell 1和Cell 2
2. 跳转到**"粘贴您的Cell 3代码"**标记处
3. 粘贴原始Cell 3代码（从 `# 全局变量存储` 开始）
4. 运行该Cell启动界面

## Cell 1: 环境设置和导入

In [ ]:
# 检测环境
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🌐 在Google Colab中运行")
    print("📥 克隆仓库...")
    !git clone https://github.com/YOUR_USERNAME/Industrial-digital-twin-by-transformer.git
    %cd Industrial-digital-twin-by-transformer
    print("📦 安装依赖...")
    !pip install -q -r requirements.txt
    print("✅ 安装完成！")
else:
    print("💻 在本地环境运行")
    print("确保已安装依赖: pip install -r requirements.txt")

## Cell 2: 导入所有必要的库和模块

In [ ]:
# ============================================================================
# 标准库导入
# ============================================================================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

# Gradio和其他工具
import gradio as gr
import json
import os
from datetime import datetime
import traceback

# Scipy（用于V4的平滑功能）
from scipy.signal import savgol_filter
from scipy.ndimage import maximum_filter1d

# ============================================================================
# 🔥 关键：导入项目模块（替代原始Cell 1和Cell 2）
# ============================================================================
from models.v1_transformer import CompactSensorTransformer
from models.v4_hybrid_transformer import HybridTemporalTransformer
from models.utils import (
    create_temporal_context_data,
    apply_ifd_smoothing,
    handle_duplicate_columns,
    get_available_signals,
    validate_signal_exclusivity_v1,
    validate_signal_exclusivity_v4
)

# ============================================================================
# 设备配置
# ============================================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"V1模型 - 使用设备: {device}")
print(f"V4模型 - 使用设备: {device}")
print("✓ V1模型定义完成")
print("✓ V4模型定义完成")
print("\n" + "="*80)
print("✅ 所有模块已成功导入！")
print("="*80)

---

## 🔴 重要提示：选择您的使用方式

### 📍 选项A：使用简化版界面

继续运行下面的cells（Cell 3-6）

### 📍 选项B：使用您的完整Cell 3代码（推荐）

1. **跳过**下面的Cells 3-6
2. **直接跳到**最后的 "粘贴您的Cell 3代码" 部分
3. 在那里粘贴您原始 `说明.txt` 的Cell 3完整代码

---

## Cell 3: 简化版Gradio界面（选项A）

如果您选择使用简化版，运行此cell。否则跳到最后。

In [ ]:
# 这是简化版，如果需要完整功能，请使用选项B（在最后一个cell）

# 全局状态
global_state = {
    'df': None,
    'trained_models': {},
    'all_signals': []
}

def load_data(file):
    """加载数据"""
    try:
        if file is None:
            return "❌ 请上传CSV文件", gr.update(choices=[])
        
        df = pd.read_csv(file.name)
        global_state['df'] = df
        
        signals = get_available_signals(df)
        global_state['all_signals'] = signals
        
        msg = f"✅ 数据加载成功！\n形状: {df.shape}\n可用信号: {len(signals)}个"
        return msg, gr.update(choices=signals)
    except Exception as e:
        return f"❌ 加载失败: {str(e)}", gr.update(choices=[])

# 创建简化的Gradio界面
with gr.Blocks(title="Industrial Digital Twin", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🚀 Industrial Digital Twin by Transformer")
    gr.Markdown("### 简化版界面 - 用于快速测试")
    
    gr.Markdown("""
    💡 **提示**: 这是简化版界面。如需完整功能（V1+V4训练、配置管理、完整推理）：
    1. 使用 `python gradio_app.py`
    2. 或在本notebook最后粘贴您的完整Cell 3代码
    """)
    
    with gr.Tab("📊 数据加载"):
        file_input = gr.File(label="上传CSV文件", file_types=[".csv"])
        load_btn = gr.Button("加载数据", variant="primary")
        status_text = gr.Textbox(label="状态", lines=5)
        
        signals_dropdown = gr.Dropdown(
            choices=[],
            multiselect=True,
            label="可用信号",
            visible=False
        )
        
        load_btn.click(
            fn=load_data,
            inputs=[file_input],
            outputs=[status_text, signals_dropdown]
        )
    
    gr.Markdown("""
    ---
    ### 📖 使用说明
    
    **完整功能请使用**:
    - `python gradio_app.py` - 命令行版本
    - 或在本notebook最后的cell粘贴完整Cell 3代码
    
    **包含的完整功能**:
    - ✅ V1静态Transformer训练
    - ✅ V4混合时序+静态训练
    - ✅ 实时训练进度
    - ✅ 配置导入/导出
    - ✅ 完整推理和可视化
    """)

print("✅ 简化版Gradio界面已准备就绪")
print("\n运行 demo.launch() 启动界面")

## Cell 4: 启动简化版界面（选项A）

In [ ]:
# 启动简化版Gradio界面
demo.launch(share=IN_COLAB, debug=True)

---

# ⭐ 完整Cell 3代码粘贴区（选项B - 推荐）

## 📋 操作步骤

### 1. 找到您的原始Cell 3代码

从您的 `说明.txt` 文件中:
- **起始位置**: 约第362行，从 `# 全局变量存储` 开始
- **结束位置**: 约第3013行，到 `demo.launch(share=True, debug=True)` 结束

### 2. 复制整段代码

包括:
- ✅ 全局变量存储 (`global_state = {...}`)
- ✅ 所有辅助函数
- ✅ 所有验证函数
- ✅ 训练函数 (`train_v1_model_complete`, `train_v4_model_complete`)
- ✅ 配置导入导出函数
- ✅ 回调函数
- ✅ Gradio界面定义 (`with gr.Blocks(...) as demo:`)
- ✅ 事件绑定
- ✅ `demo.launch()`

### 3. 粘贴到下面的cell中

### 4. 运行cell启动完整界面

---

## 💡 提示

- **不需要修改**任何代码！直接粘贴即可。
- **Cell 1和Cell 2已经导入**了模型，所以您的Cell 3可以直接使用。
- 如果粘贴后有错误，检查是否完整复制了所有代码。

---

## ⬇️ 在下面的cell中粘贴您的完整Cell 3代码 ⬇️

In [ ]:
# ============================================================================
# 🔴 粘贴您的完整Cell 3代码到这里
# ============================================================================
# 
# 从说明.txt第362行开始复制:
# 
# # Cell 3: Gradio完整界面（优化版 - 分离V1/V4 + 互斥信号选择）
# 
# import gradio as gr
# import json
# ...
# 
# 一直复制到:
# 
# demo.launch(share=True, debug=True)
# 
# ============================================================================

# 👇 粘贴位置（删除这行注释，粘贴您的代码）



# 👆 粘贴位置（删除这行注释，粘贴您的代码）

# ============================================================================
# 粘贴完成后，运行此cell启动完整Gradio界面
# ============================================================================

---

## ✅ 完成！

运行上面的cell后，您的完整Gradio界面将启动，包含:

### 功能列表
- ✅ **数据加载Tab**: 上传CSV、查看所有信号
- ✅ **V1训练Tab**: 完整V1静态Transformer训练
- ✅ **V4训练Tab**: 完整V4混合时序+静态训练
- ✅ **推理Tab**: 多模型推理、可视化分析
- ✅ **配置管理**: JSON导入/导出
- ✅ **实时进度**: 每个epoch的训练状态
- ✅ **信号验证**: 自动检查信号选择错误

### 访问界面
- **Colab**: 会生成公开链接
- **本地**: 访问 http://127.0.0.1:7860

---

## 🆘 需要帮助？

查看文档:
- `GRADIO_README.md` - Gradio使用总指南
- `docs/CELL3_INTEGRATION_GUIDE.md` - 详细集成说明
- `docs/GRADIO_INTEGRATION.md` - Gradio集成文档

---